In [10]:
import nltk
import re
import pandas as pd
import json
import os

# Télécharger les ressources NLTK
print("Téléchargement des ressources NLTK...")
nltk.download('punkt_tab', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
print("Ressources téléchargées avec succès!")

from nltk.tokenize import word_tokenize

Téléchargement des ressources NLTK...
Ressources téléchargées avec succès!


In [11]:
#Configuration des chemains d'acces vers les dossier des images

DATA_DIR = '../dataset'
TRAIN_IMAGES_DIR = os.path.join(DATA_DIR, 'train2017')      
VAL_IMAGES_DIR = os.path.join(DATA_DIR, 'val2017')         
ANNOTATIONS_FILE = os.path.join(DATA_DIR, 'annotations', 'instances_train2017.json')  #fichier qui contient les details et captions des images


#parametre : nouvelle taille des images

TARGET_SIZE = (256, 256)

In [16]:
# Fonction de nettoyage et de tokenisation des captions

def clean_caption(caption): 

    caption = caption.lower()                                #minuscule
    caption = re.sub(r'[^a-z0-9\s]', '', caption)            #supression des caracteres non aplhanumeriques
    caption = re.sub(r'\s+', ' ', caption).strip()           #remplacement de multiples espaces en un seul


    return caption
    

In [13]:
#Fonction de pretraitement des images

def preprocess_image(filepath, target_size=TARGET_SIZE):    

    try: 
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)                             #lire en echelle de gris
        
        if img is None: 
           print(f"Erreur de chargement de l'image : {filepath}")
           return None

        resized_img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

        return resized_img
        
        
    except Exception as e:
        print(f"Erreur lors du traitement de l'image {filepath}: {e}")
        return None
        

In [22]:
# Construction du vocabulaire
def process_captions(annotations_file):
    with open(annotations_file, 'r') as f:
        #lire le fichier json
        coco_json = json.load(f)
    
    category_dict = {cat['id']: cat['name'] for cat in coco_json['categories']}          #dictionnaire : cat-id : cat-name
    
    image_data = {}                                                                      #dictionnaire : img-id : caption
    for img_meta in coco_json['images']:
        image_data[img_meta['id']] = {
            'filename': img_meta['file_name'],
            'cleaned_captions': []
        }
    
    all_words = []
    
    for annotation in coco_json['annotations']:
        image_id = annotation['image_id']
        category_id = annotation['category_id']
        
        # Recuperer le nom de la catégorie
        raw_caption = category_dict.get(category_id, "unknown")
        cleaned_caption = clean_caption(raw_caption)
        
        # Vérifier si la légende nettoyée existe déjà pour cette image
        if cleaned_caption and cleaned_caption not in image_data[image_id]['cleaned_captions']:
            image_data[image_id]['cleaned_captions'].append(cleaned_caption)
            all_words.append(cleaned_caption)
    
    # Reformatage en DataFrame pour une manipulation facile (en dehors de la boucle!)
    df_captions = pd.DataFrame([
        {
            'image_id': img_id,
            'filename': data['filename'],
            'captions': data['cleaned_captions']  # ← Suffit largement !
        }
        for img_id, data in image_data.items()
        if data['cleaned_captions']
    ])
    
    print(f"Nombre total d'images avec légendes : {len(df_captions)}")
    print(f"Nombre total de tokens (avant filtrage du vocabulaire) : {len(all_words)}")
    
    # Construction d'un vocabulaire filtré (Liste de tous les mots uniques)
    vocabulary = sorted(list(set(all_words)))
    print(f"Taille du vocabulaire brut (mots uniques) : {len(vocabulary)}")
    
    return df_captions, vocabulary              #vocabulary est une liste des categories differentes
                                                #df_captions est le dataset a utiliser

In [23]:
# --- EXÉCUTION DU PRÉTRAITEMENT TEXTUEL ---
df_train_captions, train_vocabulary = process_captions(ANNOTATIONS_FILE)


print("\n--- Résultat du Prétraitement Textuel ---")
print(df_train_captions.head())

print("\n--- vocabulaire : ---")
print(train_vocabulary)


Nombre total d'images avec légendes : 5000
Nombre total de tokens (avant filtrage du vocabulaire) : 14569
Taille du vocabulaire brut (mots uniques) : 80

--- Résultat du Prétraitement Textuel ---
   image_id          filename  \
0      5802  000000005802.jpg   
1     61181  000000061181.jpg   
2     86320  000000086320.jpg   
3    463836  000000463836.jpg   
4     52759  000000052759.jpg   

                                            captions  
0       [bottle, person, knife, bowl, cup, backpack]  
1  [bicycle, motorcycle, bus, truck, traffic ligh...  
2                                             [sink]  
3                         [person, backpack, toilet]  
4                          [airplane, person, truck]  

--- vocabulaire : ---
['airplane', 'apple', 'backpack', 'banana', 'baseball bat', 'baseball glove', 'bear', 'bed', 'bench', 'bicycle', 'bird', 'boat', 'book', 'bottle', 'bowl', 'broccoli', 'bus', 'cake', 'car', 'carrot', 'cat', 'cell phone', 'chair', 'clock', 'couch', 'cow'

In [24]:
#sauvegarde du dataframe traite et le vocabulaire pour utilisation dans les prochaines etapes

#1) dataframe
df_train_captions.to_json('df_train_captions.json', orient='records', indent=2)

# 2) vocabulaire
with open('train_vocabulary.json', 'w', encoding='utf-8') as f:
    json.dump(train_vocabulary, f, indent=2, ensure_ascii=False)

print("✅ Données sauvegardées en JSON !")

✅ Données sauvegardées en JSON !
